# obt_customers_header

Esse notebook é responsável por fazer a união de informações básicas sobre os clientes do e-commerce


## Configurações Iniciais

### Preparação do Ambiente PySpark

#### Instalando as Dependências

In [27]:
# Instala o Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixa o Apache Spark 3.4.1 com Hadoop 3 (versão estável e recente)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompacta o arquivo baixado
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Instala a biblioteca findspark
!pip install -q findspark

#### Settando as Variáveis de Ambiente

In [28]:
import os

# Define a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Define a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

from pyspark.sql import SparkSession

# Cria a SparkSession
spark = SparkSession.builder.master("local[*]").appName("E-commerce Dataset").getOrCreate()

### Importando as Bibliotecas


In [29]:
from pyspark.sql import functions as F
import pandas as pd

### Carregando os Dados

In [30]:
df_customers_dataset = (
    spark.read.csv(
        "/content/olist_customers_dataset.csv",
        header=True,
        inferSchema=True
    )
)

df_customers_geolocations = (
    spark.read.csv(
        "/content/olist_geolocation_dataset.csv",
        header=True,
        inferSchema=True
    )
    .withColumnRenamed("geolocation_zip_code_prefix", "customer_zip_code_prefix")
)

## Transformação dos Dados

In [31]:
df_coordenadas_geograficas = (
    df_customers_dataset
    .join(
        df_customers_geolocations,
        "customer_zip_code_prefix",
        how="Left"
    )
)

## Salvamento dos Dados

In [33]:
(
    df_customers_dataset
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .csv("/transformed_data/obt_customers_dataset")
)